In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import os
from data import Data

C:\Users\Armughan.Shahid\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\Armughan.Shahid\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [47]:
class Params():#used to store parameter values
    def __init__(self,params):
        #STORING PARAMETER VALUES 
        self.input_shape=params['input_shape']
        self.num_outputs=params['num_outputs']
        self.layer_hierarchy=params['layer_hierarchy']
        self.activation_fn=params.get('activation_fn',tf.nn.relu)
        self.loss_fn=params.get('loss_fn',tf.losses.softmax_cross_entropy)
        self.learning_rate=params['learning_rate']
        self.optimizer_fn=params['optimizer_fn']
        self.initializer_fn=params['initializer_fn']
        self.name_scope=params['name_scope']
        logdir=params['logdir']
        self.step_no=0

In [48]:
class CNN_Model():
    def __init__(self,param_dict={},restore_params=False,pickle_file_path=""):
        #STORING PARAMETER VALUES
        if not restore_params:
            self.params=Params(param_dict)
        else:
            self.restore_params_fn(pickle_file_path)

    def form_placeholder(self,shape,dt=tf.float32):
        X=tf.placeholder(dt,shape=shape)
        return X
    def form_convolutional_layer(self,inputs,layer_params):
        return tf.layers.conv2d(
                    inputs=inputs,
                    filters=layer_params['num_filters'],
                    kernel_size=layer_params['kernel_size'],
                    strides=layer_params['kernel_strides'],
                    padding=layer_params['padding'],
                    kernel_initializer=self.params.initializer_fn(),
                    activation=None)
    def form_max_pooling_layer(self,inputs,layer_params):
        tf.layers.max_pooling2d(
                    inputs=inputs,
                    pool_size=layer_params['pool_size'],
                    strides=layer_params['pool_strides'])
    def form_activation_layer(self,inputs):
        return self.params.activation_fn(inputs)
    
    def form_fc_layer(self,inputs,layer_params):
        return tf.layers.dense(inputs,layer_params['num_hidden_units'],activation=None,kernel_initializer=self.params.initializer_fn())
    
    def form_loss(self,logits,targets):
        entropies=self.params.loss_fn(onehot_labels=targets,logits=logits)
        return entropies
    
    def build_model(self):
        with tf.variable_scope(self.params.name_scope):
            self.X=self.form_placeholder(self.params.input_shape)
            self.Y=self.form_placeholder((None,self.params.num_outputs),tf.float32)
            self.lr_placeholder=self.form_placeholder([]) #since we can change learning arate during training
            inputs=self.X
            for layer_params in self.params.layer_hierarchy:
                if layer_params['layer_type']=='conv_layer':
                    inputs=self.form_convolutional_layer(inputs,layer_params)
                elif layer_params['layer_type']=='fc_layer':
                    inputs=self.form_fc_layer(inputs,layer_params)
                elif layer_params['layer_type']=='activation_layer':
                    inputs=self.form_activation_layer(inputs)
                elif layer_params['layer_type']=='pooling_layer':
                    inputs=self.form_max_pooling_layer(inputs,layer_params)
                elif layer_params['layer_type']=='flattening_layer':
                    inputs=tf.contrib.layers.flatten(inputs)

    #         making logits layer (final output layer)
            self.logits=tf.layers.dense(inputs,self.params.num_outputs,activation=None,kernel_initializer=self.params.initializer_fn())

            self.loss=self.form_loss(self.logits,self.Y)
            optimizer=self.params.optimizer_fn(learning_rate=self.lr_placeholder)
            self.train_op=optimizer.minimize(loss = self.loss)
            
            self.initializer=tf.global_variables_initializer()
        model_variables=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.params.name_scope)#saving only the varuiables belonging to this scope
        self.saver=tf.train.Saver(var_list=model_variables)
        
    def save_model(self,sess,savedir="/"):
        step=self.params.step_no
        savedir=os.path.join(os.getcwd(),savedir)
        if not hasattr(self,'saved_before'):#calling save model for the first time
            if not os.path.isdir(savedir):#creating directory if not exists
                try:  
                    os.makedirs(savedir)
                except OSError:
                    print ('failed to make the specified_directory.Returning...')
                    return
            file_pi = open(savedir+"\\"+self.params.name_scope+".pkl", 'wb+') #saving param object
            pickle.dump(self.params, file_pi)
            #saving tensorflow graph and weight values
            self.saver.save(sess,savedir+"\\"+self.params.name_scope+".ckpt", global_step=step) #saving model weights
            self.saved_before=True
        else:    #saving model weights
            self.saver.save(sess,savedir+"\\"+self.params.name_scope+".ckpt", global_step=step,write_meta_graph=False)#writes meta graph for the first time save_model is called
    def restore_params_fn(self,pickle_file_path):
        if os.path.exists(pickle_file_path):
            filehandler = open(pickle_file_path, 'rb')
            self.params=pickle.load(filehandler)
        else:
            print("no such file exists")
        
    def restore_model(self,sess,restore_dir):
        restore_dir=os.path.join(os.getcwd(),restore_dir)
        print (restore_dir+"\\"+self.params.name_scope)
        self.saver.restore(sess, tf.train.latest_checkpoint(restore_dir+"\\"))#loading latest model
        
    def train(self,sess,n_epochs,n_iter,get_next_batch_fn,save_every_n_iter,log_every_n_iter,save_dir,initialize=False):
        if initialize:
            sess.run([self.initializer])
        for epoch in np.arange(n_epochs):
            print("----Epoch="+str(epoch)+"\n")
            it=0
            for x,y in get_next_batch_fn():
                it+=1
                if it>=n_iter:
                    break
                feed_dict={self.X:x,self.Y:y,self.lr_placeholder:self.params.learning_rate}
                loss,_=sess.run([self.loss,self.train_op],feed_dict=feed_dict)
                self.params.step_no+=1
                if (self.params.step_no)%save_every_n_iter==0:
                    print("saving model\n")
                    self.save_model(sess,save_dir)
#                 print ("""loss= "+str(loss)+"\n")
                print ("Step={} and loss occured= {} \n".format(str(self.params.step_no),str(loss)))


In [49]:
params={
    'input_shape':[None, 35, 190, 1],
    'num_outputs':3,
    
    'layer_hierarchy':[
        {'layer_type':'conv_layer','kernel_size':4,'kernel_strides':1,'num_filters':32,'padding':'valid'},
        {'layer_type':'activation_layer'},
        {'layer_type':'conv_layer','kernel_size':8,'kernel_strides':1,'num_filters':16,'padding':'valid'},
        {'layer_type':'activation_layer'},
        {'layer_type':'flattening_layer'},
        {'layer_type':'fc_layer','num_hidden_units':256},
        {'layer_type':'activation_layer'},
        {'layer_type':'fc_layer','num_hidden_units':100},
        {'layer_type':'activation_layer'},
        
    ],
    'initializer_fn':tf.contrib.layers.variance_scaling_initializer,
    'activation_fn':tf.nn.relu,
    'loss_fn':tf.losses.softmax_cross_entropy,
    'learning_rate':0.001,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/',
    'name_scope':'neural_network_v1'
}

In [20]:
# #testing model 
# tf.reset_default_graph()
# model=CNN_Model(params)
# model.build_model()
# with tf.Session() as sess:
#     sess.run(model.initializer)
#     model.save_model(sess,"1st_try")

In [21]:
# #testing restoring a new model
# tf.reset_default_graph()
# model=CNN_Model(restore_params=True,pickle_file_path="1st_try/neural_network_v1.pkl")
# model.build_model()
# with tf.Session() as sess:
#     model.restore_model(sess,"1st_try")
# #     sess.run(model.initializer)
# #     model.save_model(sess,"1st_try")

In [8]:
batch_size=120
data=Data('training_data.npy',batch_size)

Data size= 9964


In [51]:
n_epochs=50
n_iter=int(data.X_train.shape[0]/data.batch_size)
save_every_n_iter=5
log_every_n_iter=20
initialize=False
tf.reset_default_graph()
# model=CNN_Model(params)

    

model=""
with tf.Session() as sess:
    if(not initialize):
        model=CNN_Model(restore_params=True,pickle_file_path="1st_try/neural_network_v1.pkl")
        model.build_model()
        model.restore_model(sess,"1st_try")
        model.params.learning_rate=0.0003
    else:
        model=CNN_Model(params)
        model.build_model()
#         model.params.step_no=765
#     sess.run(model.initializer)
#     model.save_model(sess,"1st_try")
    model.train(sess,n_epochs,n_iter,data.get_next_batch,save_every_n_iter,log_every_n_iter,save_dir="1st_try",initialize=initialize)
#     for x,y in data.get_next_batch():
#         print (x.shape)
#         print (y.shape)

D:\dino game\AI_Plays_Dino_Game\1st_try\neural_network_v1
INFO:tensorflow:Restoring parameters from D:\dino game\AI_Plays_Dino_Game\1st_try\neural_network_v1.ckpt-15
----Epoch=0

Step=6 and loss occured= 1.0755359 

Step=7 and loss occured= 1.0725709 

Step=8 and loss occured= 1.0731826 

Step=9 and loss occured= 1.0714436 

saving model

Step=10 and loss occured= 1.0672069 



KeyboardInterrupt: 